In [9]:
#LEGACY CODE 1

import pyaudio
import keyboard
import time
import wave

chunk = 1024
format = pyaudio.paInt16
channels = 2
rate = 44100
Output_Filename = "Recorded.wav"

p = pyaudio.PyAudio()

stream = p.open(format=format,
                channels=channels,
                rate=rate,
                input=True,
                frames_per_buffer=chunk)

frames = []
print("Press SPACE to start recording")
keyboard.wait('space')
print("Recording... Press SPACE to stop.")
time.sleep(0.2)

while True:
    try:
        data = stream.read(chunk)
        frames.append(data)
    except KeyboardInterrupt:
        break
    if keyboard.is_pressed('space'):
        print("stopping recording")
        time.sleep(0.2)
        break

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(Output_Filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(format))
wf.setframerate(rate)
wf.writeframes(b''.join(frames))
wf.close()


Press SPACE to start recording


KeyboardInterrupt: 

In [ ]:
#LEGACY CODE 2
import soundcard as sc
import soundfile as sf

OUTPUT_FILE_NAME = "out.wav"    # output file name.
SAMPLE_RATE = 48000              # [Hz]. sampling rate.
RECORD_SEC = 5                  # [sec]. recording duration.

with sc.get_microphone(id=str(sc.default_speaker().name), include_loopback=True).recorder(samplerate=SAMPLE_RATE) as mic:
    # record audio with loopback from default speaker.
    data = mic.record(numframes=SAMPLE_RATE*RECORD_SEC)
    sf.write(file=OUTPUT_FILE_NAME, data=data[:, 0], samplerate=SAMPLE_RATE)






In [ ]:
import soundcard as sc
import soundfile as sf
import keyboard
import numpy as np

OUTPUT_FILE_NAME = "out.wav"    # Output file name.
SAMPLE_RATE = 48000             # [Hz], sampling rate.
chunk_size = 1024               # Size of audio chunks to record per iteration.

# Buffer to store the recorded data
frames = []

print("Press SPACE to start recording...")
keyboard.wait('space')  # Wait for the first press of spacebar to start recording
print("Recording... Press SPACE again to stop.")

# Open loopback microphone to capture system audio
with sc.get_microphone(id=str(sc.default_microphone().name), include_loopback=True).recorder(samplerate=SAMPLE_RATE) as mic:
    
    while True:
        # Record a chunk of data
        data = mic.record(numframes=chunk_size)
        frames.append(data)
        
        # Check if the spacebar is pressed again to stop the recording
        if keyboard.is_pressed('space'):
            print("Stopping recording...")
            break

# Convert the recorded frames to a NumPy array
recorded_data = np.concatenate(frames, axis=0)

# Save the recorded data to the output file
sf.write(file=OUTPUT_FILE_NAME, data=recorded_data[:, 0], samplerate=SAMPLE_RATE)

print(f"Recording saved to {OUTPUT_FILE_NAME}")


Press SPACE to start recording...
Recording... Press SPACE again to stop.
Stopping recording...
Recording saved to out.wav


In [13]:
import soundcard as sc
import soundfile as sf
import keyboard
import numpy as np
import warnings

# Suppress specific warnings related to data discontinuity
warnings.filterwarnings("ignore", category=sc.SoundcardRuntimeWarning)

# Output filenames
mic_output_file = "microphone_output.wav"
sys_output_file = "system_output.wav"

SAMPLE_RATE = 48000  # [Hz], sampling rate.
chunk_size = 4096    # Increased size to reduce handling frequency

# Buffers to store the recorded data
mic_frames = []
sys_frames = []

print("Press SPACE to start recording microphone and system audio...")
keyboard.wait('space')  # Wait for the first press of spacebar to start recording
print("Recording... Press SPACE again to stop.")

# Get the default microphone and default speaker (loopback for system audio)
microphone = sc.default_microphone()
speaker = sc.get_microphone(id=str(sc.default_speaker().name), include_loopback=True)

# Open both the microphone and system audio streams
with microphone.recorder(samplerate=SAMPLE_RATE, blocksize=chunk_size) as mic_recorder, \
     speaker.recorder(samplerate=SAMPLE_RATE, blocksize=chunk_size) as sys_recorder:

    while True:
        try:
            # Record a chunk of data from both microphone and system audio
            mic_data = mic_recorder.record(numframes=chunk_size)
            sys_data = sys_recorder.record(numframes=chunk_size)

            # Append the recorded chunks to their respective buffers
            mic_frames.append(mic_data)
            sys_frames.append(sys_data)

        except sc.SoundcardRuntimeWarning as e:
            print(f"Warning caught: {e}")
            continue  # Ignore the warning and continue recording

        # Check if the spacebar is pressed again to stop the recording
        if keyboard.is_pressed('space'):
            print("Stopping recording...")
            break

# Convert the recorded frames to a NumPy array
mic_recorded_data = np.concatenate(mic_frames, axis=0)
sys_recorded_data = np.concatenate(sys_frames, axis=0)

# Save the recorded data to output files
sf.write(file=mic_output_file, data=mic_recorded_data[:, 0], samplerate=SAMPLE_RATE)
sf.write(file=sys_output_file, data=sys_recorded_data[:, 0], samplerate=SAMPLE_RATE)

print(f"Microphone recording saved to {mic_output_file}")
print(f"System audio recording saved to {sys_output_file}")
 

Press SPACE to start recording microphone and system audio...
Recording... Press SPACE again to stop.
Stopping recording...
Microphone recording saved to microphone_output.wav
System audio recording saved to system_output.wav
